In [19]:
import pandas as pd
import numpy as np
import tensorflow as tf
colnames = ['CIC0', 'SM1_Dz(Z)', 'GATS1i', 'NdsCH', 'NdssC','MLOGP', 'LC50']
data = pd.read_csv('./data/qsar_fish_toxicity.csv', sep=';', names=colnames)
X = data.drop('LC50', axis=1)
y = data['LC50']

In [20]:
# Define the Keras models
from keras.models import Sequential
from keras.layers import Dense
def build_model_1():
    # build the Keras model_1
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model
def build_model_2():
    # build the Keras model_2
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model
def build_model_3():
    # build the Keras model_3
    model = Sequential()
    model.add(Dense(4, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(2, activation='relu'))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer='adam')
    # return the model
    return model

In [21]:
seed = 1
np.random.seed(seed)
tf.set_random_seed(seed)
# perform cross-validation on each model
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
results_1 = []
models = [build_model_1, build_model_2, build_model_3]
# loop over three models
for m in range(len(models)):
    model = KerasRegressor(build_fn=models[m], epochs=100, batch_size=20, verbose=0, shuffle=False)
    kf = KFold(n_splits=3)
    result = cross_val_score(model, X, y, cv=kf)
    print(f"Cross-Validation Loss for Model =  {abs(result.mean()):.4f}")

Cross-Validation Loss for Model =  1.0306
Cross-Validation Loss for Model =  0.9583
Cross-Validation Loss for Model =  1.7225


In [24]:
np.random.seed(seed)
tf.set_random_seed(seed)
results_2 = []
epochs = [100, 150]
batches = [20, 15]
# Loop over pairs of epochs and batch_size
for e in range(len(epochs)):
    for b in range(len(batches)):
        model = KerasRegressor(build_fn= build_model_2, epochs= epochs[e], batch_size= batches[b], verbose=0, shuffle=False)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        print(f"batch_size = {batches[b]}, epochs = {epochs[e]}, Test Loss = {abs(result.mean()):.4f}")

batch_size = 20, epochs = 100, Test Loss = 0.9575
batch_size = 15, epochs = 100, Test Loss = 0.9480
batch_size = 20, epochs = 150, Test Loss = 0.8898
batch_size = 15, epochs = 150, Test Loss = 0.9507


In [23]:
def build_model(activation='relu', optimizer='adam'):
    # build the Keras model_2
    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation=activation))
    model.add(Dense(1))
    # Compile the model
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    # return the model
    return model

activations = ['relu', 'tanh']
optimizers = ['sgd', 'adam', 'rmsprop']
#Define a seed for the random number generator so the result will be reproducible
np.random.seed(seed)
tf.set_random_seed(seed)
# Loop over pairs of activation and optimizer
for o in range(len(optimizers)):
    for a in range(len(activations)):
        optimizer = optimizers[o]
        activation = activations[a]
        model = KerasRegressor(build_fn= build_model,epochs=100, batch_size=20, verbose=0, shuffle=False)
        kf = KFold(n_splits=3)
        result = cross_val_score(model, X, y, cv=kf)
        print(f"activation = {activation}, optimizer = {optimizer}, Test Loss = {abs(result.mean()):.4f}")

activation = relu, optimizer = sgd, Test Loss = 0.9575
activation = tanh, optimizer = sgd, Test Loss = 0.9584
activation = relu, optimizer = adam, Test Loss = 0.9005
activation = tanh, optimizer = adam, Test Loss = 0.9858
activation = relu, optimizer = rmsprop, Test Loss = 0.9365
activation = tanh, optimizer = rmsprop, Test Loss = 0.9717
